In [ ]:
import datetime
import numpy as np
import os
import pandas as pd
import random
import shortuuid

from scheduler.order_scheduler import OrderScheduler

In [ ]:
ROOT_DIR = '/Users/frost/Desktop/scheduling_input/stress'

ORDER_FILE_NAME = 'demand_order.csv'
SOURCING_RULE_FILE_NAME = 'sourcing_rule.csv'
SUPPLY_PLAN_FILE_NAME = 'supply.csv'

In [ ]:
order_df = pd.read_csv(os.path.join(ROOT_DIR, ORDER_FILE_NAME))
sourcing_rule_df = pd.read_csv(os.path.join(ROOT_DIR, SOURCING_RULE_FILE_NAME))
supply_plan_df = pd.read_csv(os.path.join(ROOT_DIR, SUPPLY_PLAN_FILE_NAME))

In [ ]:
def fulfillment_plan(order_df, sourcing_rule_df, supply_plan_df):
    df_legal = lambda df, titles: set([*df.columns]) == set(titles)  and len(np.where(pd.isnull(df))[0]) == 0
    assert df_legal(order_df, ['customer', 'product', 'date', 'quantity']) and df_legal(sourcing_rule_df, ['site', 'product', 'customer']) and df_legal(supply_plan_df, ['site', 'product', 'date', 'quantity'])
    order_df['date'], supply_plan_df['date'] = pd.to_datetime(order_df['date']), pd.to_datetime(supply_plan_df['date'])
    order_df, supply_plan_df = order_df.sort_values(by=['date']), supply_plan_df.sort_values(by=['date'])
    order_scheduler = OrderScheduler()
    for _, row in sourcing_rule_df.iterrows():
        order_scheduler.add_sourcing_rule(row['customer'], row['site'], row['product'])
    fulfillment_plans = []
    for date in sorted(set(order_df['date']).union(set(supply_plan_df['date']))):
        order_daily_df, supply_daily_df = order_df[order_df['date'] == date] , supply_plan_df[supply_plan_df['date'] == date]
        for _, row in order_daily_df.iterrows():
            order_scheduler.claim_order(row['customer'], row['product'], row['quantity'], row['date'])
        for _, row in supply_daily_df.iterrows():
            order_scheduler.claim_supply_plan(row['site'], row['product'], row['quantity'], row['date'])
        fulfillment_plans += order_scheduler.plan_fulfillment(date)
    return fulfillment_plans, order_scheduler.order_queue_manager.queued_orders

In [ ]:
plans, _ = fulfillment_plan(order_df, sourcing_rule_df, supply_plan_df)

In [ ]:
order_fulfill_lookup = {}
for plan in plans:
    order_key = (plan[0], plan[1], plan[2])
    if order_key not in order_fulfill_lookup:
        order_fulfill_lookup[order_key] = []
    order_fulfill_lookup[order_key].append(plan)

In [ ]:
order_wait_times = [(max([*map(lambda x: x[4], plans)]) - key[2]).days for key, plans in order_fulfill_lookup.items()]
sum(order_wait_times) / len(order_wait_times)

### same day orders test case generation

##### average wait time metrics
- our algorithm: 191.28
- algorithm without SJF strategy: 274.16

In [ ]:
test_input_root = '/Users/frost/Desktop'

order_quantities = [np.random.randint(0, 10000) for _ in range(100)]
base_date = datetime.datetime(2020, 1, 1)

with open(os.path.join(test_input_root, 'order.txt'), 'w') as order_out_file:
    order_out_file.write('customer,product,date,quantity\n')
    for index, quantity in enumerate(order_quantities):
        order_out_file.write('C' + str(index) + ',P001,' + base_date.strftime('%d-%b-%Y') + ',' + str(quantity) + '\n')
with open(os.path.join(test_input_root, 'supply.txt'), 'w') as supply_out_file:
    supply_out_file.write('site,product,date,quantity\n')
    for index in range(sum(order_quantities) // 1000 + 1):
        supply_out_file.write('1206,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',1000\n')
with open(os.path.join(test_input_root, 'rule.txt'), 'w') as rule_out_file:
    rule_out_file.write('site,customer,product\n')
    for index in range(len(order_quantities)):
        rule_out_file.write('1206,C' + str(index) + ',P001\n')

### blocking order test case generation

##### average wait time metrics
- our algorithm: 60.47
- algorithm without supply leak: 75.76

In [ ]:
test_input_root = '/Users/frost/Desktop'

order_quantities = [5000 if np.random.randint(50) == 0 else 100 for _ in range(100)]
base_date = datetime.datetime(2020, 1, 1)

with open(os.path.join(test_input_root, 'order.txt'), 'w') as order_out_file:
    order_out_file.write('customer,product,date,quantity\n')
    for index, quantity in enumerate(order_quantities):
        order_out_file.write('C' + str(index) + ',P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',' + str(quantity) + '\n')
with open(os.path.join(test_input_root, 'supply.txt'), 'w') as supply_out_file:
    supply_out_file.write('site,product,date,quantity\n')
    for index in range(sum(order_quantities) // 100 + 1):
        supply_out_file.write('1206,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',100\n')
with open(os.path.join(test_input_root, 'rule.txt'), 'w') as rule_out_file:
    rule_out_file.write('site,customer,product\n')
    for index in range(len(order_quantities)):
        rule_out_file.write('1206,C' + str(index) + ',P001\n')

### weak origin test case generation

##### average wait time metrics
- our algorithm: 10.195
- algorithm without origin order: 10.34

In [ ]:
test_input_root = '/Users/frost/Desktop'

base_date = datetime.datetime(2020, 1, 1)

with open(os.path.join(test_input_root, 'order.txt'), 'w') as order_out_file:
    order_out_file.write('customer,product,date,quantity\n')
    for index, quantity in enumerate(range(100)):
        order_out_file.write('C1,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',100\n')
        order_out_file.write('C2,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',50\n')
with open(os.path.join(test_input_root, 'supply.txt'), 'w') as supply_out_file:
    supply_out_file.write('site,product,date,quantity\n')
    for index in range(150 * 100 // 100 + 1):
        supply_out_file.write('1206,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',75\n')
    for index in range(150 * 100 // 50 + 1):
        supply_out_file.write('1205,P001,' + (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y') + ',50\n')
with open(os.path.join(test_input_root, 'rule.txt'), 'w') as rule_out_file:
    rule_out_file.write('site,customer,product\n')
    rule_out_file.write('1206,C1,P001\n')
    rule_out_file.write('1205,C1,P001\n')
    rule_out_file.write('1205,C2,P001\n')

### stress test case generation

##### average wait time metrics
- our algorithm: 128.0156286813791
- pure FCFS algorithm: 128.0156286813791

In [ ]:
test_input_root = '/Users/frost/Desktop'

base_date = datetime.datetime(2020, 1, 1)
site_names = [shortuuid.uuid() for _ in range(100)]
customer_names = [shortuuid.uuid() for _ in range(100)]

total_quantity = 0

with open(os.path.join(test_input_root, 'order.txt'), 'w') as order_out_file:
    order_out_file.write('customer,product,date,quantity\n')
    for index, quantity in enumerate(range(256)):
        date_string = (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y')
        for customer_name in customer_names:
            if np.random.randint(2) > 0:
                quantity = np.random.randint(50, 1000)
                order_out_file.write(customer_name + ',P1,' + date_string + ',' + str(quantity) + '\n')
                total_quantity += quantity
with open(os.path.join(test_input_root, 'supply.txt'), 'w') as supply_out_file:
    supply_out_file.write('site,product,date,quantity\n')
    for site_name in site_names:
        supply_quantity, date_index = 0, 0
        while supply_quantity < total_quantity:
            date_string = (base_date + datetime.timedelta(days=index)).strftime('%d-%b-%Y')
            if np.random.randint(4) > 0:
                quantity = np.random.randint(500, 5000)
                supply_out_file.write(site_name + ',P1,' + date_string + ',' + str(quantity) + '\n')
                supply_quantity += quantity
            date_index += 1
with open(os.path.join(test_input_root, 'rule.txt'), 'w') as rule_out_file:
    rule_out_file.write('site,customer,product\n')
    for site_name in site_names:
        rule_out_file.write(site_name + ',default_customer,P1\n')
    for customer_name in customer_names:
        sites = random.choices(site_names, k=np.random.randint(2, 5))
        for site in sites:
            rule_out_file.write(site + ',' + customer_name + ',P1\n')